In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
import os
from pathlib import Path

from getAttentionLib import ActivationPatchingResult, cluster_logits_diffs

root = Path("vqa_patching")
n_img_tokens = 256
clustered_logit_diffs = []
for p in [root / p for p in os.listdir(root)]:
    pr = ActivationPatchingResult.load(p)
    cld = cluster_logits_diffs(pr.get_logit_diff(), n_img_tokens=n_img_tokens)
    clustered_logit_diffs.append(cld)
clustered_logit_diffs = torch.stack(clustered_logit_diffs)
clustered_logit_diffs.shape

In [ ]:
from getAttentionLib import plot_img_and_text_probs_side_by_side, plot_pooled_probs_plt


fig = plot_pooled_probs_plt(
    pooled_probs=clustered_logit_diffs.mean(dim=0).T,
    inputs_tokens=["img tokens", "bos token", "text tokens", "final token"],
    cbar_label="Logit Diff",
    # cmax=(cmax := clustered_logit_diffs.abs().max()),
    # cmin=-cmax,
    cmap="RdBu",
    n_img_tokens=1,
    figsize=(6, 3),
    cmin=-0.7,
    cmax=0.7,
    fraction=0.1,
    aspect=3
)
fig.tight_layout()
# fig.savefig("imgs/vqa_patching/aggregate_logit_diffs.png")